# Explore Obspy

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from obspy.clients.fdsn import Client
from obspy.core.utcdatetime import UTCDateTime
from shapely.geometry import Point, Polygon


import init
import whaletracks.common. constants as cn
from common_python.database import database_util as util
import obspy

## Constants

## ObsPy
channel data: sampling_rate, startdate, enddate, gain, poles, zero, azimuth, dip, channel type, sensor,
sensitivity, gain1 (stage 1 gain), gain2, gain3

In [2]:
def getStations(network_code, starttime= UTCDateTime("2001-01-01"), 
                endtime=UTCDateTime("2020-01-02")):
    """
    Provides a dataframe of station metadata.
    :param str network_code:
    :return pd.DataFrame:
        station_code, channel_count, start_time, end_time, access, latitude, longitude, elevation
    """
    KEYS = [
     'code',
     'creation_date',
     'elevation',
     'end_date',
     'latitude',
     'longitude',
     'start_date',
     'termination_date',
     'total_number_of_channels'
    ]
    client = Client("IRIS")
    inventory = client.get_stations(network=network_code, station="*",
        starttime=starttime, endtime=endtime)
    network = inventory[0]
    station_count = network.total_number_of_stations
    station_dct = {k: [] for k in KEYS}
    station_dct["network"] = list(np.repeat(network_code, station_count))
    for idx in range(station_count):
        station = network[idx]
        for key in KEYS:
            station_dct[key].append(station.__getattribute__(key))
    if False:
        station_lines = network.get_contents()["stations"]
        station_codes = []
        for line in station_lines:
            splits = line.split(" ")
            station_codes.append(splits[0][-4:])
        return station_codes
    return pd.DataFrame(station_dct)



station_df = getStations("7D")

In [75]:

starttime = UTCDateTime("2010-01-01")

endtime = UTCDateTime("2020-01-02")
client = Client("IRIS")

inventory = client.get_stations(network="7D", station="*", channel="*Z", level="response",

                                starttime=starttime,

                                endtime=endtime)
#print(inventory)

if False:
    client = Client("IRIS")

    t1 = UTCDateTime("2010-02-27T06:30:00.000")

    t2 = t1 + 1

    t3 = t1 + 3

    bulk = [("IU", "ANMO", "*", "BHZ", t1, t2),

            ("IU", "AFI", "1?", "BHE", t1, t3),

            ("GR", "GRA1", "*", "BH*", t2, t3)]

    inv = client.get_stations_bulk(bulk, level="channel")
    print(inv)  

In [ ]:
def getNetwork(network="7D", channel="*Z", starttime=UTCDateTime("2010-01-01"),
                 endtime=UTCDateTime("2020-01-01")):
    client = Client("IRIS")
    inventory = client.get_stations(network=network, station="*", channel=channel,
        level="response", starttime=starttime, endtime=endtime)
    return inventory[0]

In [166]:
def makeChannelID(network_code, station_code, channel_code):
    return "%s.%s.%s" % (network_code, station_code, channel_code)

def makeStationID(network_code, station_code):
    return "%s.%s" % (network_code, station_code)

def getPolesZeroes(channel):
    try:
        zeroes = ";".join([str((x.real, x.imag)) for x in channel.response.get_paz().zeros])
    except:
        zeroes = ""
    try:
        poles = ";".join([str((x.real, x.imag)) for x in channel.response.get_paz().poles])
    except:
        poles = ""
    return poles, zeroes
# Add codes to convert poles to a list of complex numbers
        

def getChannelMetadata(network):
    """
    :param obspy.inventory.network.Network network:
    :return pd.DataFrame: channels, poles, zeros
      channel dataframe:
        station_code, channel_code,
        sampling_rate, startdate, enddate, gain, poles, zeroes, azimuth, dip, channel type, sensor,
        sensitivity
      poles, zeros
          
    """
    NETWORK_CODE = "network_code"
    STATION_ID = "station_id"  # NETWORK_CODE.STATION_CODE
    CHANNEL_ID = "channel_id"  # NETWORK_CODE.STATION_CODE.CHANNEL_CODE
    CHANNEL_TYPES = "channel_types"
    START_DATE = "start_date"
    END_DATE = "end_date"
    GAIN = "gain"
    POLES = "poles"  # semicolon separated values of immaginary numbers
    AZIMUTH = "azimuth"
    DIP = "dip"
    CHANNEL_TYPE = "channel_type"
    SENSOR = "sensor"
    SENSITIVITY_FREQUENCY = "sensitivity_frequency"
    SENSITIVITY_VALUE = "sensitivity_value"
    VALUE = "value"
    ZEROES = "zeroes" # semicolon separated values of immaginary numbers
    COLUMNS = [
        AZIMUTH, CHANNEL_ID, CHANNEL_TYPES, DIP, END_DATE, 
        POLES, SENSITIVITY_FREQUENCY, 
        SENSITIVITY_VALUE, SENSOR, START_DATE, STATION_ID, ZEROES,
    ]

    channel_dct = {k: [] for k in COLUMNS}
    for station in network:
        station_id = makeStationID(network.code, station.code)
        for channel in station:
            poles, zeroes = getPolesZeroes(channel)
            channel_dct[AZIMUTH].append(channel.azimuth)
            channel_dct[CHANNEL_ID].append(makeChannelID(network.code, station.code, channel.code))
            channel_dct[CHANNEL_TYPES].append(" ".join(channel.types))
            channel_dct[DIP].append(channel.dip)
            channel_dct[END_DATE].append(channel.end_date)
            channel_dct[POLES].append(poles)
            channel_dct[SENSITIVITY_FREQUENCY].append(channel.response.instrument_sensitivity.frequency)
            channel_dct[SENSITIVITY_VALUE].append(channel.response.instrument_sensitivity.value)
            channel_dct[SENSOR].append(channel.sensor.description)
            channel_dct[START_DATE].append(channel.start_date)
            channel_dct[STATION_ID].append(makeStationID(network.code, station.code))
            channel_dct[ZEROES].append(zeroes)
    return pd.DataFrame(channel_dct)

In [164]:
df = getChannelMetadata(net)
df.head()

,azimuth,channel_id,channel_types,dip,end_date,poles,sensitivity_frequency,sensitivity_value,sensor,start_date,station_id,zeroes
0,0.0,7D.FC03D.HHZ,GEOPHYSICAL,-90.0,2015-09-03T23:11:06.000000Z,"(-0.03691, 0.03712);(-0.03691, -0.03712);(-371...",1.0,250883000.0,Nanometrics Trillium Compact Response/LDEO OBS...,2014-09-07T21:48:20.000000Z,7D.FC03D,"(0.0, 0.0);(0.0, 0.0);(-434.1, 0.0)"
1,0.0,7D.FC03D.HXZ,GEOPHYSICAL,-90.0,2015-09-03T23:10:38.000000Z,"(-0.03691, 0.03712);(-0.03691, -0.03712);(-371...",1.0,250883000.0,Nanometrics Trillium Compact Response/LDEO OBS...,2014-09-07T21:48:20.000000Z,7D.FC03D,"(0.0, 0.0);(0.0, 0.0);(-434.1, 0.0)"
2,0.0,7D.FN01A.HHZ,GEOPHYSICAL,-90.0,2012-07-21T00:00:17.000000Z,"(-0.03691, 0.03712);(-0.03691, -0.03712);(-371...",1.0,971188000.0,Nanometrics Trillium Compact Response/LDEO OBS...,2011-07-27T20:15:00.000000Z,7D.FN01A,"(0.0, 0.0);(0.0, 0.0);(-434.1, 0.0)"
3,0.0,7D.FN01C.HHZ,GEOPHYSICAL,-90.0,2014-06-27T05:00:17.000000Z,"(-0.03691, 0.03712);(-0.03691, -0.03712);(-371...",1.0,971188000.0,Nanometrics Trillium Compact Response/LDEO OBS...,2013-08-31T16:14:00.000000Z,7D.FN01C,"(0.0, 0.0);(0.0, 0.0);(-434.1, 0.0)"
4,0.0,7D.FN01C.HXZ,GEOPHYSICAL,-90.0,2014-06-27T05:00:17.000000Z,"(-0.03691, 0.03712);(-0.03691, -0.03712);(-371...",1.0,971188000.0,Nanometrics Trillium Compact Response/LDEO OBS...,2013-08-30T16:14:00.000000Z,7D.FN01C,"(0.0, 0.0);(0.0, 0.0);(-434.1, 0.0)"


In [165]:
pole_strs = df.loc[0, "poles"].split(";")
pole_strs

['(-0.03691, 0.03712)',
 '(-0.03691, -0.03712)',
 '(-371.2, 0.0)',
 '(-373.9, 475.5)',
 '(-373.9, -475.5)',
 '(-588.4, 1508.0)',
 '(-588.4, -1508.0)']

In [ ]:
net = inventory[0]
stations = [s for s in net if len(station.channels) > 0 ]
len(stations)

In [66]:
station

Station FC03D (LDEO OBS TRAWL-RESISTANT)
	Station Code: FC03D
	Channel Count: 2/9 (Selected/Total)
	2014-09-07T00:00:00.000000Z - 2015-10-02T23:59:59.000000Z
	Access: open 
	Latitude: 44.81, Longitude: -124.74, Elevation: -432.0 m
	Available Channels:
		FC03D..HHZ, FC03D..HXZ

In [111]:
station = net[0]
channel = station[0]
channel

'Nanometrics Trillium Compact Response/LDEO OBS 201'

In [76]:
channel.response

Channel Response
	From M/S (velocity in meters per second) to COUNTS (digital counts)
	Overall Sensitivity: 2.50883e+08 defined at 1.000 Hz
	3 stages:
		Stage 1: PolesZerosResponseStage from M/S to V, gain: 747.682
		Stage 2: PolesZerosResponseStage from V to V, gain: 335548
		Stage 3: CoefficientsTypeResponseStage from V to COUNTS, gain: 1

In [109]:
channel.response.instrument_sensitivity.value

250883000.0

In [98]:
dir(channel.response)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_eval_resp_for_frequencies',
 '_get_overall_sensitivity_and_gain',
 '_repr_pretty_',
 'get_evalresp_response',
 'get_evalresp_response_for_frequencies',
 'get_paz',
 'get_sacpz',
 'instrument_polynomial',
 'instrument_sensitivity',
 'plot',
 'recalculate_overall_sensitivity',
 'resource_id',
 'response_stages']

In [144]:
dir(channel.response.get_paz())

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_normalization_frequency',
 '_poles',
 '_pz_transfer_function_type',
 '_repr_pretty_',
 '_zeros',
 'decimation_correction',
 'decimation_delay',
 'decimation_factor',
 'decimation_input_sample_rate',
 'decimation_offset',
 'description',
 'input_units',
 'input_units_description',
 'name',
 'normalization_factor',
 'normalization_frequency',
 'output_units',
 'output_units_description',
 'poles',
 'pz_transfer_function_type',
 'resource_id',
 'resource_id2',
 'stage_gain',
 'stage_gain_frequency',
 'stage_sequence_number',
 'zeros']

In [162]:
x = channel.response.get_paz().poles
x[0].real

-0.03691

In [6]:
net.stations

[Station FC03D (LDEO OBS TRAWL-RESISTANT)
	Station Code: FC03D
	Channel Count: 0/9 (Selected/Total)
	2014-09-07T00:00:00.000000Z - 2015-10-02T23:59:59.000000Z
	Access: open 
	Latitude: 44.81, Longitude: -124.74, Elevation: -432.0 m
	Available Channels:
,
 Station FN01A (LDEO Trawl-Resistant OBS Site FN01A)
	Station Code: FN01A
	Channel Count: 0/5 (Selected/Total)
	2011-07-27T00:00:00.000000Z - 2012-07-21T23:59:59.000000Z
	Access: open 
	Latitude: 46.88, Longitude: -124.33, Elevation: -54.0 m
	Available Channels:
,
 Station FN01C (LDEO OBS TRAWL-RESISTANT)
	Station Code: FN01C
	Channel Count: 0/9 (Selected/Total)
	2013-08-30T00:00:00.000000Z - 2014-06-27T23:59:59.000000Z
	Access: open 
	Latitude: 46.88, Longitude: -124.33, Elevation: -56.0 m
	Available Channels:
,
 Station FN02C (LDEO OBS TRAWL-RESISTANT)
	Station Code: FN02C
	Channel Count: 0/9 (Selected/Total)
	2013-08-31T00:00:00.000000Z - 2014-06-28T23:59:59.000000Z
	Access: open 
	Latitude: 46.95, Longitude: -124.43, Elevation: -67

In [7]:
station = net[0]; station

Station FC03D (LDEO OBS TRAWL-RESISTANT)
	Station Code: FC03D
	Channel Count: 0/9 (Selected/Total)
	2014-09-07T00:00:00.000000Z - 2015-10-02T23:59:59.000000Z
	Access: open 
	Latitude: 44.81, Longitude: -124.74, Elevation: -432.0 m
	Available Channels:


In [39]:
station.channels

[]

In [8]:
str(station.creation_date)

'2014-09-07T00:00:00.000000Z'

In [9]:
station.latitude, station.longitude

(44.813301, -124.738297)

In [36]:
station.channels

[]

## Acquiring Channel Data

In [10]:
starttime = UTCDateTime("2001-01-01")
endtime = UTCDateTime("2020-01-02")
client = Client("IRIS")
inventory = client.get_stations(network="7D", station="*",
                                starttime=starttime,
                                endtime=endtime)

In [11]:
stations = [net[n] for n in range(net.total_number_of_stations) if len(net[n].channels) > 0]

In [12]:
net[2]

Station FN01C (LDEO OBS TRAWL-RESISTANT)
	Station Code: FN01C
	Channel Count: 0/9 (Selected/Total)
	2013-08-30T00:00:00.000000Z - 2014-06-27T23:59:59.000000Z
	Access: open 
	Latitude: 46.88, Longitude: -124.33, Elevation: -56.0 m
	Available Channels:


In [13]:
net

Network 7D (Cascadia Initiative Community Experiment - OBS Component)
	Station Count: 259/259 (Selected/Total)
	2011-01-01T00:00:00.000000Z - 2017-12-31T23:59:59.000000Z
	Access: open
	Contains:
		Stations (259):
			7D.FC03D (LDEO OBS TRAWL-RESISTANT)
			7D.FN01A (LDEO Trawl-Resistant OBS Site FN01A)
			7D.FN01C (LDEO OBS TRAWL-RESISTANT)
			7D.FN02C (LDEO OBS TRAWL-RESISTANT)
			7D.FN03A (LDEO Trawl-Resistant OBS Site FN03A)
			7D.FN03C (LDEO OBS TRAWL-RESISTANT)
			7D.FN04C (LDEO OBS TRAWL-RESISTANT)
			7D.FN05A (LDEO OBS TRAWL-RESISTANT)
			7D.FN05C (LDEO OBS TRAWL-RESISTANT)
			7D.FN06A (LDEO Trawl-Resistant OBS Site FN06A)
			7D.FN06C (LDEO OBS TRAWL-RESISTANT)
			7D.FN07A (LDEO Trawl-Resistant OBS Site FN07A)
			7D.FN07C (LDEO OBS TRAWL-RESISTANT)
			7D.FN08A (LDEO Trawl-Resistant OBS Site FN08A)
			7D.FN08C (LDEO OBS TRAWL-RESISTANT)
			7D.FN09A (LDEO Trawl-Resistant OBS Site FN09A)
			7D.FN09C (LDEO OBS TRAWL-RESISTANT)
			7D.FN10A (LDEO Trawl-Resistant OBS Site FN10A)
			7D.FN

In [14]:
UTCDateTime("2013-01-01") + 3600

2013-01-01T01:00:00.000000Z

In [15]:
HOUR = 3600
starttime = UTCDateTime("2013-01-01")
endtime = starttime + 0.5*HOUR
st_raw = client.get_waveforms(network="7D", station="*", location = "*",
                              channel="BHZ,HHZ", starttime=starttime, endtime=endtime)

In [16]:
HOUR = 3600
def getChannelMetadata(network_code="7D", starttime=UTCDateTime("2013-01-01"),
                      endtime=UTCDateTime("2013-01-01")+HOUR):
    """
    Obtains sampling rate information for channels.
    :return pd.DataFrame:
        network, station, channel, sampling_rate
    """
    NETWORK = "network"
    STATION = "station"
    CHANNEL = "channel"
    SAMPLING_RATE = "sampling_rate"
    COLUMNS = [NETWORK, STATION, CHANNEL, SAMPLING_RATE]
    st_raw = client.get_waveforms(network=network_code, station="*", location = "*",
                              channel="BHZ,HHZ", starttime=starttime, endtime=endtime)
    idx = 0
    dct = {k: [] for k in COLUMNS}
    while True:
        try:
            stats = st_raw[idx].stats
        except IndexError:
            break
        dct[STATION].append(stats.station)
        dct[NETWORK].append(stats.network)
        dct[CHANNEL].append(stats.channel)
        dct[SAMPLING_RATE].append(stats.sampling_rate)
        idx += 1
    return dct

In [22]:
dct = getChannelMetadata()

In [26]:
len(dct["channel"])

67

In [18]:
dir(st_raw[1].stats)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__class__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_fdsnws_dataselect_url',
 '_format',
 '_pretty_str',
 '_repr_pretty_',
 'calib',
 'channel',
 'clear',
 'copy',
 'defaults',
 'delta',
 'do_not_warn_on',
 'endtime',
 'get',
 'items',
 'keys',
 'location',
 'mseed',
 'network',
 'npts',
 'pop',
 'popitem',
 'readonly',
 'sampling_rate',
 'setdefault',
 'starttime',
 'station',
 'update',
 'values',
 'warn_on_non_default_k

In [19]:
st_raw[2].stats

               network: 7D
               station: FS03B
              location: 
               channel: HHZ
             starttime: 2013-01-01T00:00:00.007800Z
               endtime: 2013-01-01T00:29:59.999800Z
         sampling_rate: 125.0
                 delta: 0.008
                  npts: 225000
                 calib: 1.0
_fdsnws_dataselect_url: http://service.iris.edu/fdsnws/dataselect/1/query
               _format: MSEED
                 mseed: AttribDict({'dataquality': 'M', 'number_of_records': 191, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 4096, 'filesize': 16814080})

In [20]:
st_raw._get_common_channels_info()

{('7D', 'J25B', '', 'HH?'): {'start': 2013-01-01T00:00:00.005900Z,
  'end': 2013-01-01T00:29:59.997900Z,
  'gaps': [],
  'channels': {'HHZ': {'start': 2013-01-01T00:00:00.005900Z,
    'end': 2013-01-01T00:29:59.997900Z}}},
 ('7D', 'FS04B', '', 'HH?'): {'start': 2013-01-01T00:00:00.001900Z,
  'end': 2013-01-01T00:29:59.993790Z,
  'gaps': [],
  'channels': {'HHZ': {'start': 2013-01-01T00:00:00.001900Z,
    'end': 2013-01-01T00:29:59.993790Z}}},
 ('7D', 'G29B', '', 'BH?'): {'start': 2013-01-01T00:00:00.004698Z,
  'end': 2013-01-01T00:29:59.984698Z,
  'gaps': [],
  'channels': {'BHZ': {'start': 2013-01-01T00:00:00.004698Z,
    'end': 2013-01-01T00:29:59.984698Z}}},
 ('7D', 'M12B', '', 'BH?'): {'start': 2013-01-01T00:00:00.003173Z,
  'end': 2013-01-01T00:29:59.983173Z,
  'gaps': [],
  'channels': {'BHZ': {'start': 2013-01-01T00:00:00.003173Z,
    'end': 2013-01-01T00:29:59.983173Z}}},
 ('7D', 'M10B', '', 'HH?'): {'start': 2013-01-01T00:00:00.002600Z,
  'end': 2013-01-01T00:29:59.991991Z,
  

In [21]:
# Can get channel gap information, but not other metadata?
st_raw._get_common_channels_info()

{('7D', 'J25B', '', 'HH?'): {'start': 2013-01-01T00:00:00.005900Z,
  'end': 2013-01-01T00:29:59.997900Z,
  'gaps': [],
  'channels': {'HHZ': {'start': 2013-01-01T00:00:00.005900Z,
    'end': 2013-01-01T00:29:59.997900Z}}},
 ('7D', 'FS04B', '', 'HH?'): {'start': 2013-01-01T00:00:00.001900Z,
  'end': 2013-01-01T00:29:59.993790Z,
  'gaps': [],
  'channels': {'HHZ': {'start': 2013-01-01T00:00:00.001900Z,
    'end': 2013-01-01T00:29:59.993790Z}}},
 ('7D', 'G29B', '', 'BH?'): {'start': 2013-01-01T00:00:00.004698Z,
  'end': 2013-01-01T00:29:59.984698Z,
  'gaps': [],
  'channels': {'BHZ': {'start': 2013-01-01T00:00:00.004698Z,
    'end': 2013-01-01T00:29:59.984698Z}}},
 ('7D', 'M12B', '', 'BH?'): {'start': 2013-01-01T00:00:00.003173Z,
  'end': 2013-01-01T00:29:59.983173Z,
  'gaps': [],
  'channels': {'BHZ': {'start': 2013-01-01T00:00:00.003173Z,
    'end': 2013-01-01T00:29:59.983173Z}}},
 ('7D', 'M10B', '', 'HH?'): {'start': 2013-01-01T00:00:00.002600Z,
  'end': 2013-01-01T00:29:59.991991Z,
  

## Alternative Collection of Channel Metadata

In [27]:
starttime = UTCDateTime("2001-01-01")

endtime = UTCDateTime("2020-01-02")
client = Client("IRIS")

inventory = client.get_stations(network="7D", station="*",

                                starttime=starttime,

                                endtime=endtime)

In [35]:
inventory.get_channel_metadata("7D.G29B..BH?")

Exception: No matching channel metadata found.